<a href="https://colab.research.google.com/github/yeonmi02/CPR_project/blob/main/KnowYourMeme_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install requests beautifulsoup4 pandas

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

base_url = "https://knowyourmeme.com"
search_terms = [
    "this is fine", "chill guy", "everything is fine", "calm meme", "unbothered", "fire around me"
]

results = []

for term in search_terms:
    print(f"🔍 Searching: {term}")
    for page in range(1, 6):  # 페이지당 약 20개, 총 100개 이상 확보
        url = f"https://knowyourmeme.com/search?page={page}&q={term.replace(' ', '+')}"
        res = requests.get(url)
        soup = BeautifulSoup(res.content, "html.parser")

        for entry in soup.select(".entry-grid-body .photo"):
            title = entry.select_one("a")["title"]
            link = base_url + entry.select_one("a")["href"]
            results.append({"title": title, "link": link})

        time.sleep(1)

df = pd.DataFrame(results).drop_duplicates()
df.to_csv("kym_chill_guy_memes.csv", index=False)
print(f"✅ 수집된 밈 수: {len(df)}개")

🔍 Searching: this is fine
🔍 Searching: chill guy
🔍 Searching: everything is fine
🔍 Searching: calm meme
🔍 Searching: unbothered
🔍 Searching: fire around me
✅ 수집된 밈 수: 0개


In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import time

# ✅ Chill Guy 관련 키워드
search_query = "this is fine"  # 또는 "chill guy"
base_url = "https://knowyourmeme.com/search"

# ✅ 수집할 페이지 수
max_pages = 20  # 20페이지 * 25개 = 최대 500개 예상

results = []

for page in tqdm(range(1, max_pages + 1), desc="🔥 Crawling KYM"):
    url = f"{base_url}?page={page}&q={search_query}"
    res = requests.get(url)
    soup = BeautifulSoup(res.content, "html.parser")

    entries = soup.select(".entry-grid-body .photo")
    if not entries:
        print(f"⚠️ No entries found on page {page}")
        break

    for entry in entries:
        try:
            title = entry.select_one("a")["title"]
            link = "https://knowyourmeme.com" + entry.select_one("a")["href"]
            results.append({"title": title, "url": link})
        except:
            continue

    time.sleep(1.0)

# ✅ 결과 DataFrame으로 변환
df = pd.DataFrame(results)
df.drop_duplicates(inplace=True)

# ✅ 저장
df.to_csv("kym_chill_guy_memes.csv", index=False)
print(f"✅ 수집 완료: {len(df)}개 저장됨")

🔥 Crawling KYM:   0%|          | 0/20 [00:01<?, ?it/s]

⚠️ No entries found on page 1
✅ 수집 완료: 0개 저장됨


In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm

base_url = "https://knowyourmeme.com/memes/this-is-fine/photos/page/"
max_pages = 20

results = []

for page in tqdm(range(1, max_pages + 1), desc="🔥 Crawling Gallery"):
    url = base_url + str(page)
    res = requests.get(url)
    soup = BeautifulSoup(res.content, "html.parser")

    images = soup.select("div.photo > a")
    if not images:
        print(f"⚠️ No images found on page {page}")
        break

    for img in images:
        try:
            link = "https://knowyourmeme.com" + img["href"]
            img_tag = img.select_one("img")
            img_url = img_tag["data-src"] if "data-src" in img_tag.attrs else img_tag["src"]
            title = img_tag["alt"]
            results.append({"title": title, "url": link, "img_url": img_url})
        except:
            continue

    time.sleep(1.0)

# ✅ 저장
df = pd.DataFrame(results)
df.drop_duplicates(inplace=True)
df.to_csv("kym_chill_guy_gallery.csv", index=False)
print(f"✅ 수집 완료: {len(df)}개 이미지")

🔥 Crawling Gallery:   0%|          | 0/20 [00:01<?, ?it/s]

⚠️ No images found on page 1
✅ 수집 완료: 0개 이미지


In [8]:
!pip install selenium webdriver-manager

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 33.0 MB/s eta 0:00:00


In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import time
from tqdm import tqdm

# ✅ 브라우저 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # 창 없이 실행
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# ✅ 크롤링 대상 URL
base_url = "https://knowyourmeme.com/memes/this-is-fine/photos/page/"

results = []

for page in tqdm(range(1, 21), desc="🔥 Crawling KYM"):
    driver.get(base_url + str(page))
    time.sleep(2)  # JS 로딩 대기

    images = driver.find_elements(By.CSS_SELECTOR, "div.photo > a")

    if not images:
        print(f"⚠️ No images found on page {page}")
        break

    for img in images:
        try:
            href = img.get_attribute("href")
            img_tag = img.find_element(By.TAG_NAME, "img")
            img_url = img_tag.get_attribute("src")
            title = img_tag.get_attribute("alt")
            results.append({"title": title, "url": href, "img_url": img_url})
        except:
            continue

driver.quit()

# ✅ 저장
df = pd.DataFrame(results)
df.drop_duplicates(inplace=True)
df.to_csv("kym_chill_guy_gallery.csv", index=False)
print(f"✅ 수집 완료: {len(df)}개 이미지")

WebDriverException: Message: unknown error: cannot find Chrome binary
Stacktrace:
#0 0x56bd0e5f34e3 <unknown>
#1 0x56bd0e322c76 <unknown>
#2 0x56bd0e349757 <unknown>
#3 0x56bd0e348029 <unknown>
#4 0x56bd0e386ccc <unknown>
#5 0x56bd0e38647f <unknown>
#6 0x56bd0e37dde3 <unknown>
#7 0x56bd0e3532dd <unknown>
#8 0x56bd0e35434e <unknown>
#9 0x56bd0e5b33e4 <unknown>
#10 0x56bd0e5b73d7 <unknown>
#11 0x56bd0e5c1b20 <unknown>
#12 0x56bd0e5b8023 <unknown>
#13 0x56bd0e5861aa <unknown>
#14 0x56bd0e5dc6b8 <unknown>
#15 0x56bd0e5dc847 <unknown>
#16 0x56bd0e5ec243 <unknown>
#17 0x7fe8ae59eac3 <unknown>


In [10]:
!apt-get update
!apt-get install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,927 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,901 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3

In [13]:
!apt-get update
!apt-get install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [14]:
import sys
import os

os.environ['PATH'] += ':/usr/bin/chromedriver'
sys.path.insert(0, '/usr/lib/chromium-browser/')

In [16]:
!apt-get update -y
!apt-get install -y chromium-browser chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [17]:
import sys
import os

os.environ['PATH'] += ':/usr/bin/chromedriver'


In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=options)


In [19]:
driver.get("https://www.google.com")
print(driver.title)  # "Google"이 출력되면 성공


Google


In [20]:
base_url = "https://knowyourmeme.com/memes/this-is-fine/photos/page/"
results = []

for page in tqdm(range(1, 21), desc="🔥 Crawling KYM"):
    driver.get(base_url + str(page))
    time.sleep(2)

    images = driver.find_elements(By.CSS_SELECTOR, "div.photo > a")
    if not images:
        print(f"⚠️ No images found on page {page}")
        break

    for img in images:
        try:
            href = img.get_attribute("href")
            img_tag = img.find_element(By.TAG_NAME, "img")
            img_url = img_tag.get_attribute("src")
            title = img_tag.get_attribute("alt")
            results.append({"title": title, "url": href, "img_url": img_url})
        except:
            continue

driver.quit()

df = pd.DataFrame(results)
df.drop_duplicates(inplace=True)
df.to_csv("kym_chill_guy_gallery.csv", index=False)
print(f"✅ 수집 완료: {len(df)}개 이미지")

🔥 Crawling KYM:   0%|          | 0/20 [00:13<?, ?it/s]

⚠️ No images found on page 1
✅ 수집 완료: 0개 이미지
